In [26]:
# Toets die pylontech seriepoort kommunikasie
import sys
import pylonpacket
import logging
import serial
import time



class PylonCom:
    PORT = "/dev/ttyUSB0"
    BAUD = 115200
    
    def __init__(self):
        self.sp = serial.Serial(PylonCom.PORT,PylonCom.BAUD,timeout=0.5)

    def GetReply(self, request, reply_type):
        self.sp.write(request.GetAsciiBytes())
        line = bytearray()
        while True:
            c = self.sp.read(1)
            if c:
                line.extend(c)
                if c[0] == 0x0D:
                    break
            else:
                break
        logging.debug("Received sentence %s",line)
        preply = pylonpacket.PylonPacket.Parse(line, reply_type)
        return preply

    def close(self):
        self.sp.close()



In [32]:
pc = PylonCom()

ppIn = pylonpacket.PPGetSystemParameter()
print("Get system parameter:",ppIn)
ppOut = pc.GetReply(ppIn, pylonpacket.PPSystemParameter)
print("Get system parameter reply:",ppOut)

Get system parameter: VER: 0x20, ADR: 0x02, CID1: 0x46, CID2: 0x47, LENGTH: 0, len(INFO): 0, CHKSUM: 0x0000
Get system parameter reply: None


In [30]:
pc = PylonCom()

ppIn = pylonpacket.PPGetSystemParameter()
print("Get system parameter:",ppIn)
#ppOut = pc.GetReply(ppIn, pylonpacket.PPSystemParameter)
#print("Get system parameter reply:",ppOut)

ppIn = pylonpacket.PPGetSeriesNumber()
ppIn.Command = 0x02
print("Get series number:",ppIn)
ppOut = pc.GetReply(ppIn, pylonpacket.PPSeriesNumber)
print("Get series number reply:",ppOut) #,ppOut.info.hex())

for adr in range(2,3):
    print("Conectiong to addr",adr)
    ppIn = pylonpacket.PPGetChargeManagementInformation()
    ppIn.Command = adr
    ppIn.ADR = adr
    #print("Get charge info:",ppIn)
    ppOut = pc.GetReply(ppIn, pylonpacket.PPChargeManagementInformation)
    print("Get charge info reply:",ppOut)
      
    #return
    ppIn = pylonpacket.PPGetAnalogValue()
    ppIn.Command = adr
    ppIn.ADR = adr
    #print("Get analog:",ppIn)
    ppOut = pc.GetReply(ppIn, pylonpacket.PPAnalogValue)
    print("Get analog reply:",ppOut)

Get system parameter: VER: 0x20, ADR: 0x02, CID1: 0x46, CID2: 0x47, LENGTH: 0, len(INFO): 0, CHKSUM: 0x0000
Get series number: VER: 0x20, ADR: 0x02, CID1: 0x46, CID2: 0x93, LENGTH: 2, len(INFO): 1, CHKSUM: 0x0000, Command: 2


ValueError: Invalid packet format

In [35]:
pc = PylonCom()

ppIn=pylonpacket.PPGetManufacturerInfo()
print("Get manufacturer info:",ppIn)
ppOut=pc.GetReply(ppIn, pylonpacket.PPManufacturerInfo)
print("Get manufacturer info reply:",ppOut)

Get manufacturer info: VER: 0x20, ADR: 0x02, CID1: 0x46, CID2: 0x51, LENGTH: 0, len(INFO): 0, CHKSUM: 0x0000


ValueError: Invalid packet format

In [36]:
PORT = "/dev/ttyUSB0"
BAUD = 115200
    
sp = serial.Serial(PORT,BAUD,timeout=0.5)

In [37]:
sp


Serial<id=0xae6c0b2c, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)